In [1]:
from ehrmonize import LLM_API

In [2]:
LLM_API('meta.llama2-70b-chat-v1').prompt('What is the capital of France? One word only.')

{'generation': ' Paris', 'prompt_token_count': 23, 'generation_token_count': 2, 'stop_reason': 'stop'}


' Paris'

In [3]:
LLM_API('mistral.mistral-7b-instruct-v0:2').prompt("What is the capital of France? One word only:")

{'outputs': [{'text': ' Paris', 'stop_reason': 'stop'}]}


In [4]:
LLM_API('gpt-3.5-turbo').prompt('What is the capital of France? One word only.')

'Paris'

In [5]:
LLM_API('gpt-4').prompt('What is the capital of France? One word only.')

'Paris'

In [6]:
LLM_API('gpt-4o').prompt('What is the capital of France? One word only.')

'Paris'